In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [2]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [3]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [4]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", nrows=10).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows,addoc_id,dist,dist_cats
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785,1136820,0.537786,1.254512
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327,1392479,0.430210,0.098995
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853,1601512,0.245120,1.304837
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426,1652347,0.259652,1.192209
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145,933594,0.124739,0.409647


In [5]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'dist', 'dist_cats', 'clicked']
#usecols = ['display_id', 'ad_id', 'clicksPerShows', 'platform', 'traffic_source', 'dist', 'clicked']
features = ['clicksPerShows', 'dist2', 'dist_cats']
#features = ['clicksPerShows', 'clicksPerShowsM']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']

In [6]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", nrows=10, usecols=usecols).head()

,display_id,ad_id,clicked,clicksPerShows,dist,dist_cats
0,8,95724,0,0.034785,0.537786,1.254512
1,8,175694,0,0.202327,0.430210,0.098995
2,8,280430,1,0.256853,0.245120,1.304837
3,8,329774,0,0.052426,0.259652,1.192209
4,2657,70081,0,0.150145,0.124739,0.409647


## Preparation for Validation

In [ ]:
train_nrows, test_nrows = 650000, 210000 #  None, None #

In [ ]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [ ]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

In [ ]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", usecols = usecols, nrows=nrows)
train.count()

In [ ]:
dist_mean = train['dist'].mean()
dist_cats_mean = train['dist_cats'].mean()
train = train.fillna({'dist':dist_mean, 'dist_cats':dist_cats_mean}, inplace=True)

In [ ]:
train['dist2'] = np.square(train['dist'])
train['dist3'] = np.square(train['dist'])*train['dist']
train['dist_cats2'] = np.square(train['dist_cats'])
train['dist_cats3'] = train['dist_cats2']*train['dist_cats']
train['clicksPerShows2'] = np.square(train['clicksPerShows'])
#train['dist_cats'] = np.sqrt(train['dist_cats'])
train.count()

In [ ]:
test = train.ix[test_index].copy()
train = train.ix[train_index].copy()
train.count()

In [ ]:
test.count()

In [ ]:
trainM = pd.DataFrame({'clicksPerShows_min':train.groupby('display_id')['clicksPerShows'].min(),
                       'clicksPerShows_max':train.groupby('display_id')['clicksPerShows'].max(),
                       'clicksPerShows2_min':train.groupby('display_id')['clicksPerShows2'].min(),
                       'clicksPerShows2_max':train.groupby('display_id')['clicksPerShows2'].max(),
                       'dist_min':train.groupby('display_id')['dist'].min(),
                       'dist_max':train.groupby('display_id')['dist'].max(),
                       'dist_cats_min':train.groupby('display_id')['dist_cats'].min(),
                       'dist_cats_max':train.groupby('display_id')['dist_cats'].max(),
                       'dist2_min':train.groupby('display_id')['dist2'].min(),
                       'dist2_max':train.groupby('display_id')['dist2'].max(),
                       'dist3_min':train.groupby('display_id')['dist3'].min(),
                       'dist3_max':train.groupby('display_id')['dist3'].max(),
                       'dist_cats2_min':train.groupby('display_id')['dist_cats2'].min(),
                       'dist_cats2_max':train.groupby('display_id')['dist_cats2'].max(),
                       'dist_cats3_min':train.groupby('display_id')['dist_cats3'].min(),
                       'dist_cats3_max':train.groupby('display_id')['dist_cats3'].max()
                      }).reset_index()
trainM.count()

In [ ]:
trainM['clicksPerShows_l'] = trainM['clicksPerShows_max'] - trainM['clicksPerShows_min']
trainM['clicksPerShows2_l'] = trainM['clicksPerShows2_max'] - trainM['clicksPerShows2_min']
trainM['dist_l'] = trainM['dist_max'] - trainM['dist_min']
trainM['dist_cats_l'] = trainM['dist_cats_max'] - trainM['dist_cats_min']
trainM['dist2_l'] = trainM['dist2_max'] - trainM['dist2_min']
trainM['dist3_l'] = trainM['dist3_max'] - trainM['dist3_min']
trainM['dist_cats2_l'] = trainM['dist_cats2_max'] - trainM['dist_cats2_min']
trainM['dist_cats3_l'] = trainM['dist_cats3_max'] - trainM['dist_cats3_min']
trainM.drop(['clicksPerShows_max', 'dist_max', 'dist_cats_max'], axis=1, inplace=True)
trainM.drop(['dist2_max', 'dist_cats2_max'], axis=1, inplace=True)
trainM.drop(['dist_cats3_max'], axis=1, inplace=True)
trainM.drop(['dist3_max'], axis=1, inplace=True)
trainM.count()

In [ ]:
print sum(trainM['clicksPerShows_l'] == 0)
print sum(trainM['clicksPerShows2_l'] == 0)
print sum(trainM['dist_l'] == 0)
print sum(trainM['dist_cats_l'] == 0)
print sum(trainM['dist2_l'] == 0)
print sum(trainM['dist3_l'] == 0)
print sum(trainM['dist_cats2_l'] == 0)
print sum(trainM['dist_cats3_l'] == 0)

In [ ]:
trainM.loc[trainM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
trainM.loc[trainM['clicksPerShows2_l'] == 0, 'clicksPerShows2_l'] = 1
trainM.loc[trainM['dist_l'] == 0, 'dist_l'] = 1
trainM.loc[trainM['dist_cats_l'] == 0, 'dist_cats_l'] = 1
trainM.loc[trainM['dist2_l'] == 0, 'dist2_l'] = 1
trainM.loc[trainM['dist3_l'] == 0, 'dist3_l'] = 1
trainM.loc[trainM['dist_cats2_l'] == 0, 'dist_cats2_l'] = 1
trainM.loc[trainM['dist_cats3_l'] == 0, 'dist_cats3_l'] = 1

In [ ]:
train = train.merge(trainM, on='display_id')
train.count()

In [ ]:
del trainM

In [ ]:
train['clicksPerShows'] = ( train['clicksPerShows'] - train['clicksPerShows_min'] ) / train['clicksPerShows_l']
train['clicksPerShows2'] = ( train['clicksPerShows2'] - train['clicksPerShows2_min'] ) / train['clicksPerShows2_l']
train['dist'] = ( train['dist'] - train['dist_min'] ) / train['dist_l']
train['dist_cats'] = ( train['dist_cats'] - train['dist_cats_min'] ) / train['dist_cats_l']
train['dist2'] = ( train['dist2'] - train['dist2_min'] ) / train['dist2_l']
train['dist3'] = ( train['dist3'] - train['dist3_min'] ) / train['dist3_l']
train['dist_cats2'] = ( train['dist_cats2'] - train['dist_cats2_min'] ) / train['dist_cats2_l']
train['dist_cats3'] = ( train['dist_cats3'] - train['dist_cats3_min'] ) / train['dist_cats3_l']

train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
train.drop(['clicksPerShows2_min', 'clicksPerShows2_l'], axis=1, inplace=True)
train.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
train.drop(['dist2_min', 'dist2_l'], axis=1, inplace=True)
train.drop(['dist3_min', 'dist3_l'], axis=1, inplace=True)
train.drop(['dist_cats2_min', 'dist_cats2_l'], axis=1, inplace=True)
train.drop(['dist_cats3_min', 'dist_cats3_l'], axis=1, inplace=True)

train.count()

In [ ]:
testM = pd.DataFrame({'clicksPerShows_min':test.groupby('display_id')['clicksPerShows'].min(),
                      'clicksPerShows_max':test.groupby('display_id')['clicksPerShows'].max(),
                      'clicksPerShows2_min':test.groupby('display_id')['clicksPerShows2'].min(),
                      'clicksPerShows2_max':test.groupby('display_id')['clicksPerShows2'].max(),
                      'dist_min':test.groupby('display_id')['dist'].min(),
                      'dist_max':test.groupby('display_id')['dist'].max(),
                      'dist_cats_min':test.groupby('display_id')['dist_cats'].min(),
                      'dist_cats_max':test.groupby('display_id')['dist_cats'].max(),
                      'dist2_min':test.groupby('display_id')['dist2'].min(),
                      'dist2_max':test.groupby('display_id')['dist2'].max(),
                      'dist3_min':test.groupby('display_id')['dist3'].min(),
                      'dist3_max':test.groupby('display_id')['dist3'].max(),
                      'dist_cats2_min':test.groupby('display_id')['dist_cats2'].min(),
                      'dist_cats2_max':test.groupby('display_id')['dist_cats2'].max(),
                      'dist_cats3_min':test.groupby('display_id')['dist_cats3'].min(),
                      'dist_cats3_max':test.groupby('display_id')['dist_cats3'].max()
                      }).reset_index()
testM.count()

In [ ]:
testM['clicksPerShows_l'] = testM['clicksPerShows_max'] - testM['clicksPerShows_min']
testM['clicksPerShows2_l'] = testM['clicksPerShows2_max'] - testM['clicksPerShows2_min']
testM['dist_l'] = testM['dist_max'] - testM['dist_min']
testM['dist_cats_l'] = testM['dist_cats_max'] - testM['dist_cats_min']
testM['dist2_l'] = testM['dist2_max'] - testM['dist2_min']
testM['dist3_l'] = testM['dist3_max'] - testM['dist3_min']
testM['dist_cats2_l'] = testM['dist_cats2_max'] - testM['dist_cats2_min']
testM['dist_cats3_l'] = testM['dist_cats3_max'] - testM['dist_cats3_min']
testM.drop(['clicksPerShows_max', 'dist_max', 'dist_cats_max'], axis=1, inplace=True)
testM.drop(['clicksPerShows2_max'], axis=1, inplace=True)
testM.drop(['dist2_max', 'dist_cats2_max', 'dist_cats3_max'], axis=1, inplace=True)
testM.drop(['dist3_max'], axis=1, inplace=True)
testM.count()

In [ ]:
print sum(testM['clicksPerShows_l'] == 0)
print sum(testM['clicksPerShows2_l'] == 0)
print sum(testM['dist_l'] == 0)
print sum(testM['dist_cats_l'] == 0)
print sum(testM['dist2_l'] == 0)
print sum(testM['dist3_l'] == 0)
print sum(testM['dist_cats2_l'] == 0)
print sum(testM['dist_cats3_l'] == 0)

In [ ]:
testM.loc[testM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
testM.loc[testM['clicksPerShows2_l'] == 0, 'clicksPerShows2_l'] = 1
testM.loc[testM['dist_l'] == 0, 'dist_l'] = 1
testM.loc[testM['dist_cats_l'] == 0, 'dist_cats_l'] = 1
testM.loc[testM['dist2_l'] == 0, 'dist2_l'] = 1
testM.loc[testM['dist3_l'] == 0, 'dist3_l'] = 1
testM.loc[testM['dist_cats2_l'] == 0, 'dist_cats2_l'] = 1
testM.loc[testM['dist_cats3_l'] == 0, 'dist_cats3_l'] = 1

In [ ]:
test = test.merge(testM, on='display_id')
del testM
test.count()

In [ ]:
test['clicksPerShows'] = ( test['clicksPerShows'] - test['clicksPerShows_min'] ) / test['clicksPerShows_l']
test['clicksPerShows2'] = ( test['clicksPerShows2'] - test['clicksPerShows2_min'] ) / test['clicksPerShows2_l']
test['dist'] = ( test['dist'] - test['dist_min'] ) / test['dist_l']
test['dist_cats'] = ( test['dist_cats'] - test['dist_cats_min'] ) / test['dist_cats_l']
test['dist2'] = ( test['dist2'] - test['dist2_min'] ) / test['dist2_l']
test['dist3'] = ( test['dist3'] - test['dist3_min'] ) / test['dist3_l']
test['dist_cats2'] = ( test['dist_cats2'] - test['dist_cats2_min'] ) / test['dist_cats2_l']
test['dist_cats3'] = ( test['dist_cats3'] - test['dist_cats3_min'] ) / test['dist_cats3_l']

test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
test.drop(['clicksPerShows2_min', 'clicksPerShows2_l'], axis=1, inplace=True)
test.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
test.drop(['dist2_min', 'dist2_l'], axis=1, inplace=True)
test.drop(['dist3_min', 'dist3_l'], axis=1, inplace=True)
test.drop(['dist_cats2_min', 'dist_cats2_l'], axis=1, inplace=True)
test.drop(['dist_cats3_min', 'dist_cats3_l'], axis=1, inplace=True)

test.count()

In [ ]:
scaler = preprocessing.StandardScaler()
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

In [ ]:
train[features].head()

## Validation

In [ ]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=True, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

In [ ]:
clf.classes_

In [ ]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

In [ ]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

In [ ]:
test[features].head()

In [ ]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

In [ ]:
clf.coef_, clf.intercept_[0]

In [ ]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

In [ ]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

In [ ]:
prediction.head()

In [ ]:
validation.head()

In [ ]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.662869594202      0.588499968978      

In [ ]:
#['clicksPerShows']                             0.662869594202       0.588499968978 
#['clicksPerShows', 'dist']                     0.66282443126        0.588638094838     -> 0.662839173432 (niter=20)
                                                                    #0.588785527156
# ['clicksPerShows', 'dist', 'dist_cats']       0.663048972212       0.589767768832    
# ['clicksPerShows', 'dist_cats']                                    0.589561035359
# ['clicksPerShows', 'dist2', 'dist_cats']                           0.589826269431
# [1, 'clicksPerShows', 'dist2', 'dist_cats']                        0.589922876476 <<<<<<<<<<
    
                                      #         0.6628185045   +platform
                                     #          0.662806944913 +intercept
#['clicksPerShows', 'dist', 'adsPerDisplay']                         0.588542728654                                     
#['clicksPerShows', 'p1', 'p2', 'p3', 'dist']                        0.588165686987
#['clicksPerShows', 'dist', 't1', 't2', 't3']                        0.588181057654
#['clicksPerShows', 'p1', 'p2', 'p3']                                0.588499968978
#['clicksPerShows', 't1', 't2', 't3']                                0.588499968978 
#['clicksPerShows', 'adsPerDisplay']                                 0.588499968978


## Preparation for Training

In [7]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv", usecols = usecols)
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         83237252
dtype: int64

In [8]:
train['dist2'] = np.square(train['dist'])
dist2_mean = train['dist2'].mean()
dist_cats_mean = train['dist_cats'].mean()
train = train.fillna({'dist2':dist2_mean, 'dist_cats':dist_cats_mean}, inplace=True)

In [9]:
trainM = pd.DataFrame({'clicksPerShows_min':train.groupby('display_id')['clicksPerShows'].min(),
                       'clicksPerShows_max':train.groupby('display_id')['clicksPerShows'].max(),
                       'dist2_min':train.groupby('display_id')['dist2'].min(),
                       'dist2_max':train.groupby('display_id')['dist2'].max(),
                       'dist_cats_min':train.groupby('display_id')['dist_cats'].min(),
                       'dist_cats_max':train.groupby('display_id')['dist_cats'].max()
                      }).reset_index()
trainM.count()

display_id            16874593
clicksPerShows_max    16874593
clicksPerShows_min    16874593
dist2_max             16874593
dist2_min             16874593
dist_cats_max         16874593
dist_cats_min         16874593
dtype: int64

In [10]:
trainM['clicksPerShows_l'] = trainM['clicksPerShows_max'] - trainM['clicksPerShows_min']
trainM['dist2_l'] = trainM['dist2_max'] - trainM['dist2_min']
trainM['dist_cats_l'] = trainM['dist_cats_max'] - trainM['dist_cats_min']
trainM.drop(['clicksPerShows_max', 'dist2_max', 'dist_cats_max'], axis=1, inplace=True)
trainM.count()

display_id            16874593
clicksPerShows_min    16874593
dist2_min             16874593
dist_cats_min         16874593
clicksPerShows_l      16874593
dist2_l               16874593
dist_cats_l           16874593
dtype: int64

In [12]:
print sum(trainM['clicksPerShows_l'] == 0)
print sum(trainM['dist2_l'] == 0)
print sum(trainM['dist_cats_l'] == 0)

 191
1668118
1268690


In [13]:
trainM.loc[trainM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
trainM.loc[trainM['dist2_l'] == 0, 'dist2_l'] = 1
trainM.loc[trainM['dist_cats_l'] == 0, 'dist_cats_l'] = 1

In [14]:
trainM.to_csv("../temp/trainM_full.csv", index=False)

In [19]:
input_file = "../generated/final/events_clicks_CTR_addocs_dist_cats_train.csv"
output_file = "../temp/train_merge.csv"
chunksize=10 * 10**6
how='inner'

In [22]:
first_time = True
size = 0
i = 0
    
f = open(output_file, 'w')

for train in pd.read_csv(input_file, chunksize=chunksize):
    train['dist2'] = np.square(train['dist'])
    train = train.fillna({'dist2':dist2_mean, 'dist_cats':dist_cats_mean}, inplace=True)
    
    train = train.merge(trainM, on='display_id')
    
    train['clicksPerShows'] = ( train['clicksPerShows'] - train['clicksPerShows_min'] ) / train['clicksPerShows_l']
    train['dist2'] = ( train['dist2'] - train['dist2_min'] ) / train['dist2_l']
    train['dist_cats'] = ( train['dist_cats'] - train['dist_cats_min'] ) / train['dist_cats_l']
    train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist2_min', 'dist2_l'], axis=1, inplace=True)
    train.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)

    if first_time:
        train.to_csv(f, index = False, header=True)
        f.close()
        f = open(output_file, 'a')
        first_time = False
    else:
        train.to_csv(f, index = False, header=False)
    
    size+=chunksize
    i+=1
    print i, size
    
    #break
            
print 'Done'
    
f.close()

1 10000000
2 20000000
3 30000000
4 40000000
5 50000000
6 60000000
7 70000000
8 80000000
9 90000000
Done


In [24]:
del trainM

In [23]:
usecols + ['dist2']

['display_id',
 'ad_id',
 'clicksPerShows',
 'dist',
 'dist_cats',
 'clicked',
 'dist2']

In [25]:
train = pd.read_csv(output_file, usecols = usecols + ['dist2'])
train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
clicksPerShows    87141731
dist              77941525
dist_cats         87141731
dist2             87141731
dtype: int64

In [26]:
scaler = {}
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler[f] = preprocessing.StandardScaler()
        scaler[f].fit(train[[f]])
        train[f] = scaler[f].transform(train[[f]]) 
        #test[f] = scaler.transform(test[[f]])

clicksPerShows
dist2
dist_cats


In [27]:
train[features].head()

,clicksPerShows,dist2,dist_cats
0,-1.123149,1.623187,0.882523
1,0.852229,0.464474,-1.304975
2,1.495111,-0.926396,0.977794
3,-0.915157,-0.844771,0.764578
4,-0.050381,-0.926396,-1.304975


### Training

In [28]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=True, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [29]:
clf.classes_

array([0, 1], dtype=int64)

In [30]:
clf.coef_, clf.intercept_[0]

(array([[ 0.92548349,  0.046745  , -0.10966562]]), -1.6855743266444816)

In [31]:
del train

### Preparation for Prediction

In [32]:
test = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_cats_test.csv", usecols = usecols[:-1])
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              27426869
dist_cats         30827903
dtype: int64

In [33]:
test['dist2'] = np.square(test['dist'])

In [34]:
test = test.fillna({'dist2':dist2_mean, 'dist_cats':dist_cats_mean}, inplace=True)
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              27426869
dist_cats         32225162
dist2             32225162
dtype: int64

In [35]:
testM = pd.DataFrame({'clicksPerShows_min':test.groupby('display_id')['clicksPerShows'].min(),
                      'clicksPerShows_max':test.groupby('display_id')['clicksPerShows'].max(),
                      'dist2_min':test.groupby('display_id')['dist2'].min(),
                      'dist2_max':test.groupby('display_id')['dist2'].max(),
                      'dist_cats_min':test.groupby('display_id')['dist_cats'].min(),
                      'dist_cats_max':test.groupby('display_id')['dist_cats'].max()
                      }).reset_index()
testM.count()

display_id            6245533
clicksPerShows_max    6245533
clicksPerShows_min    6245533
dist2_max             6245533
dist2_min             6245533
dist_cats_max         6245533
dist_cats_min         6245533
dtype: int64

In [36]:
testM['clicksPerShows_l'] = testM['clicksPerShows_max'] - testM['clicksPerShows_min']
testM['dist2_l'] = testM['dist2_max'] - testM['dist2_min']
testM['dist_cats_l'] = testM['dist_cats_max'] - testM['dist_cats_min']
testM.drop(['clicksPerShows_max', 'dist2_max', 'dist_cats_max'], axis=1, inplace=True)

testM.count()

display_id            6245533
clicksPerShows_min    6245533
dist2_min             6245533
dist_cats_min         6245533
clicksPerShows_l      6245533
dist2_l               6245533
dist_cats_l           6245533
dtype: int64

In [37]:
print sum(testM['clicksPerShows_l'] == 0)
print sum(testM['dist2_l'] == 0)
print sum(testM['dist_cats_l'] == 0)

5545
894325
476289


In [38]:
testM.loc[testM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
testM.loc[testM['dist2_l'] == 0, 'dist2_l'] = 1
testM.loc[testM['dist_cats_l'] == 0, 'dist_cats_l'] = 1

In [39]:
test = test.merge(testM, on='display_id')
del testM
test.count()

display_id            32225162
ad_id                 32225162
clicksPerShows        32225162
dist                  27426869
dist_cats             32225162
dist2                 32225162
clicksPerShows_min    32225162
dist2_min             32225162
dist_cats_min         32225162
clicksPerShows_l      32225162
dist2_l               32225162
dist_cats_l           32225162
dtype: int64

In [40]:
test['clicksPerShows'] = ( test['clicksPerShows'] - test['clicksPerShows_min'] ) / test['clicksPerShows_l']
test['dist2'] = ( test['dist2'] - test['dist2_min'] ) / test['dist2_l']
test['dist_cats'] = ( test['dist_cats'] - test['dist_cats_min'] ) / test['dist_cats_l']
test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist2_min', 'dist2_l'], axis=1, inplace=True)
test.drop(['dist_cats_min', 'dist_cats_l'], axis=1, inplace=True)
#test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              27426869
dist_cats         32225162
dist2             32225162
dtype: int64

In [41]:
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        test[f] = scaler[f].transform(test[[f]])  #Use scaler from train

clicksPerShows
dist2
dist_cats


In [42]:
test[features].head()

,clicksPerShows,dist2,dist_cats
0,-0.596455,-0.701766,-1.304975
1,-0.092357,-0.365801,-1.304975
2,-1.123149,-0.346452,-1.304975
3,1.495111,1.623187,-1.304975
4,-0.706363,-0.582469,-1.304975


### Prediction

In [43]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicksPerShows,dist,dist_cats,dist2,res
0,16874904,51910,-0.596455,0.125380,-1.304975,-0.701766,0.106474
1,16874904,151689,-0.092357,0.185765,-1.304975,-0.365801,0.161781
2,16874904,156459,-1.123149,0.188655,-1.304975,-0.346452,0.069260
3,16874904,165380,1.495111,0.381760,-1.304975,1.623187,0.479290
4,16874904,256392,-0.706363,0.149638,-1.304975,-0.582469,0.097667


In [44]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dist              27426869
dist_cats         32225162
dist2             32225162
res               32225162
dtype: int64

In [45]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

display_id    6245533
ad_id         6245533
dtype: int64

In [46]:
prediction.head()

,display_id,ad_id
0,16874594,170392 172888 162754 150083 180797 66758
1,16874595,8846 143982 30609
2,16874596,289122 289915 132820 11430 173005 288385 57197...
3,16874597,285834 305790 143981 182039 308836 155945 1809...
4,16874598,145937 335632 67292 250082


### Submission

In [47]:
prediction.to_csv("../submissions/solution_9.csv", index=False)